# Тонкая настройка модели GPT-2

Этот ноутбук демонстрирует процесс тонкой настройки модели GPT-2 на пользовательском наборе данных в формате JSON Lines. Включает установку зависимостей, загрузку данных, токенизацию, обучение модели и сохранение результатов. После обучения предоставляется интерактивный виджет для генерации текста с использованием тонко настроенной модели.

In [ ]:
# Установка необходимых библиотек
!pip install "transformers>=4.41.0"
!pip install datasets torch>=2.0.0 tensorboard ipywidgets

In [ ]:
import os
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

if not os.path.exists("data"):
    os.makedirs("data")
with open("data/my_corpus.jsonl", "w", encoding="utf-8") as f:
    f.write('{"text": "Пример текста для обучения модели."}\n')
    f.write('{"text": "Еще одна строка текста для демонстрации."}\n')
    f.write('{"text": "Тонкая настройка GPT-2 на пользовательских данных."}\n')

In [ ]:
import json
from datasets import Dataset, DatasetDict

# Читаем данные из файла вручную
data = []
with open("data/my_corpus.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

# Создаем Dataset из списка словарей
raw_dataset = Dataset.from_dict({"text": [item["text"] for item in data]})

# Создаем DatasetDict с разделом 'train'
dataset = DatasetDict({
    'train': raw_dataset
})

print("Данные успешно загружены вручную.")
print(dataset)

In [ ]:
# Инициализация токенизатора и модели
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Установка pad_token равным eos_token, чтобы избежать предупреждений
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [ ]:
# Функция токенизации
def tokenize(example):
    tokenized = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

# Токенизация набора данных
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=['text'])

In [ ]:
# Определение аргументов для обучения
training_args = TrainingArguments(
    output_dir="./model",
    logging_dir="./logs",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    logging_strategy="steps",
    save_strategy="steps",
    eval_strategy="no",
    load_best_model_at_end=False,
    report_to="tensorboard"
)

In [ ]:
# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

In [ ]:
# Начало обучения
trainer.train()

In [ ]:
# Сохранение финальной модели и токенизатора
model.save_pretrained("./model/final")
tokenizer.save_pretrained("./model/final")

print("Обучение завершено. Модель и токенизатор сохранены в ./model/final")

## Интерактивная генерация текста

Следующий блок кода создает интерактивный виджет для генерации текста с использованием тонко настроенной модели GPT-2. Вы можете ввести начальный текст (промпт), настроить параметры генерации и получить результат.

In [ ]:
# Импорт необходимых библиотек для виджетов
import ipywidgets as widgets
from ipywidgets import Textarea, IntSlider, FloatSlider, Checkbox, Button, Output
from IPython.display import display
import torch

# Функция для генерации текста
def generate_text(prompt, max_length=50, temperature=0.8, top_k=50, do_sample=True):
    # Токенизация входного промпта
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    # Генерация текста
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        do_sample=do_sample,
        pad_token_id=tokenizer.eos_token_id,
        attention_mask=input_ids.ne(tokenizer.pad_token_id)
    )
    
    # Декодирование результата
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Интерактивный виджет для ввода текста
text_input = Textarea(
    value="Пример текста для генерации",
    placeholder="Введите ваш промпт здесь",
    description="Промпт:",
    layout={'width': '500px', 'height': '100px'}
)

max_length_slider = IntSlider(value=50, min=10, max=200, step=10, description="Макс. длина:")
temperature_slider = FloatSlider(value=0.8, min=0.1, max=1.5, step=0.1, description="Температура:")
top_k_slider = IntSlider(value=50, min=10, max=100, step=10, description="Top K:")
do_sample_checkbox = Checkbox(value=True, description="Использовать выборку")

output_area = Output()

def on_generate_button_clicked(b):
    with output_area:
        output_area.clear_output()
        result = generate_text(
            prompt=text_input.value,
            max_length=max_length_slider.value,
            temperature=temperature_slider.value,
            top_k=top_k_slider.value,
            do_sample=do_sample_checkbox.value
        )
        print("Сгенерированный текст:")
        print(result)

generate_button = Button(description="Сгенерировать текст")
generate_button.on_click(on_generate_button_clicked)

# Отображение виджетов
display(text_input)
display(max_length_slider)
display(temperature_slider)
display(top_k_slider)
display(do_sample_checkbox)
display(generate_button)
display(output_area)